In [39]:
import sys
sys.path.append("..")

from app.analyzer import analyze_entry
from app.storage import save_entry, load_entries

In [1]:
!pip install --upgrade streamlit

  Using cached streamlit-1.45.1-py3-none-any.whl.metadata (8.9 kB)
Using cached streamlit-1.45.1-py3-none-any.whl (9.9 MB)
  Attempting uninstall: streamlit
    Found existing installation: streamlit 1.37.1
    Uninstalling streamlit-1.37.1:
      Successfully uninstalled streamlit-1.37.1


In [41]:
journal_text = """
I’ve been feeling overwhelmed by the pace of work lately. Even when I finish something, the to-do list just keeps growing. 
I know I should be proud of what I accomplish, but I can't help feeling like I'm falling short.
"""


In [43]:
result = analyze_entry(journal_text)
save_entry(journal_text, result)


✅ Entry saved at 2025-06-01T23:51:44.313319


In [44]:
entries = load_entries()
entries[-1]


{'text': "\nI’ve been feeling overwhelmed by the pace of work lately. Even when I finish something, the to-do list just keeps growing. \nI know I should be proud of what I accomplish, but I can't help feeling like I'm falling short.\n",
 'analysis': {'summary': 'Feeling overwhelmed by the pace of work which never seems to end.',
  'emotions': ['Overwhelmed', 'Disappointment'],
  'patterns': ['Catastrophizing', 'Self-Blame', 'Comparison'],
  'themes': ['Work Stress'],
  'ai_thoughts': "Okay, so I need to analyze this journal entry and figure out the summary, emotions, cognitive distortions, and core themes. Let me read it carefully.\n\nThe person says they're overwhelmed by the pace of work. Even after finishing something, the list keeps growing. They know they should be proud but feel like they're falling short.\n\nFirst, summarizing: The main point is feeling overwhelmed with work tasks that never seem to end.\n\nEmotions: Overwhelmed and disappointment make sense because they can't f

In [ ]:
from app.storage import load_entries

entries = load_entries()

# Find the entry by keyword (e.g., 'new Journal')
for i, e in enumerate(reversed(entries)):
    if "new Journal" in e["text"]:
        print(f"Found at index: {-1 - i}")
        print(e["text"])
        break


In [ ]:
entry = entries[-2]
print("📝 Text:\n", entry["text"])
print("🧠 Summary:\n", entry["summary"])
print("🧾 Raw Output:\n", entry.get("raw_output", "No raw output found"))


In [ ]:
from app.analyzer import analyze_entry

In [ ]:
result = analyze_entry(entry['text'])


In [ ]:
result

In [ ]:
# generate_realistic_journals.py
# Uses DeepSeek (via Ollama) to create 150 diverse journal entries

import json
import random
import time
from datetime import datetime, timedelta
import requests

# Constants
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "deepseek-r1"
OUTPUT_PATH = "/Users/Additional Storage/ML Projects/Journal app/data/generated_deepseek_entries.jsonl"

# Emotion/theme seeds for diversity
emotions_pool = [
    "anxiety", "joy", "loneliness", "hope", "confusion", "pride", "grief", "boredom", "motivation",
    "excitement", "fear", "regret", "curiosity", "burnout", "peace"
]
topics_pool = [
    "relationships", "work stress", "family dynamics", "career growth", "mental health", "self-discovery",
    "routine fatigue", "health journey", "romantic feelings", "loss", "uncertainty about future",
    "celebrating a win", "dealing with rejection", "spiritual questioning"
]

# Prompt to generate realistic journal text
ENTRY_GENERATION_PROMPT = """
You're a human writing a private journal.
Your tone should be raw, informal, and personal.
Write a journal entry about feeling {emotion} today, with thoughts related to {topic}.
Include emotion, thought loops, and sensory detail if helpful.
Keep it between 4–8 sentences.
Do not include headings or lists.
Just return the journal text.
"""

import re

# Generate a single journal text from deepseek
def generate_journal_text(emotion, topic):
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a human journaling assistant."},
            {"role": "user", "content": ENTRY_GENERATION_PROMPT.format(emotion=emotion, topic=topic)}
        ],
        "stream": False
    }
    try:
        response = requests.post(OLLAMA_URL, json=payload)
        raw = response.json()["message"]["content"].strip()
        cleaned = re.sub(r"<think>.*?</think>", "", raw, flags=re.DOTALL).strip()
        return cleaned
    except Exception as e:
        print(f"❌ Failed to generate entry for {emotion} + {topic}: {e}")
        return None

# Generate realistic timestamp
def random_timestamp(start_year=2024, end_year=2025):
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    total_days = (end_date - start_date).days
    random_days = random.randint(0, total_days)
    random_seconds = random.randint(0, 86400)
    return (start_date + timedelta(days=random_days, seconds=random_seconds)).isoformat()

# Generate entries and save
entries = []
combos = list(set((e, t) for e in emotions_pool for t in topics_pool))
random.shuffle(combos)

total = 2
print(f"🧠 Generating {total} entries via DeepSeek…")

for i in range(total):
    emotion, topic = combos[i % len(combos)]
    text = generate_journal_text(emotion, topic)
    if not text:
        continue

    entry = {
        "timestamp": random_timestamp(),
        "text": text,
        "summary": "",  # To be filled by your analyzer
        "emotions": [],
        "patterns": [],
        "themes": [],
        "ai_thoughts": "",
        "raw_output": ""
    }
    entries.append(entry)
    print(f"✅ {i+1}/{total}: {emotion}, {topic}")
    time.sleep(1.5)  # Optional pacing if DeepSeek needs time

# Save to .jsonl
with open(OUTPUT_PATH, "w") as f:
    for entry in entries:
        f.write(json.dumps(entry) + "\n")

print(f"\n✅ Done. Saved {len(entries)} entries to: {OUTPUT_PATH}")


In [ ]:
entries = []

with open(OUTPUT_PATH, "r") as f:
    for line in f:
        entries.append(json.loads(line.strip()))

# Now you can access individual ones like:
print(entries[0]["text"])


In [ ]:
entries = []

with open(OUTPUT_PATH, "r") as f:
    for line in f:
        entries.append(json.loads(line.strip()))

# Now you can access individual ones like:
print(entries[0])


In [ ]:
entries = []

with open(OUTPUT_PATH, "r") as f:
    for line in f:
        entries.append(json.loads(line.strip()))

# Now you can access individual ones like:
print(entries[5]['text'])


In [ ]:
# generate_realistic_journals.py
import json, random, re, requests
from datetime import datetime, timedelta
from pathlib import Path

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "deepseek-r1"
OUTPUT_PATH = "/Users/Additional Storage/ML Projects/Journal app/data/test_generated_entries.jsonl"
TOTAL_ENTRIES = 100
TEMPERATURE = 0.8

emotions = ["anxiety", "joy", "confusion", "hope", "frustration", "nostalgia", "burnout", "contentment", "loneliness"]
topics = ["work stress", "relationships", "self-doubt", "mental health", "growth", "grief", "career uncertainty", "romantic confusion", "burnout recovery", "loneliness", "change and transition", "creative block", "financial pressure", "family expectations", "isolation", "identity", "body image", "success anxiety", "fear of failure", "disconnection from others", "social anxiety"]

GEN_PROMPT = """
You are a private journal writer.
Write a thoughtful, emotionally realistic journal entry from someone who is experiencing the emotion: {emotion}, in the context of: {topic}.
The tone should feel personal and introspective — not scripted / cmputer written, it should sound like a normal decently educated indian english writer.
Write 4–8 flowing sentences, including sensory details and authentic reflection.
Ensure good grammar and spelling, and avoid artificial phrasing or markdown.
Return only the journal entry text with no commentary or extra formatting.
"""

def generate_entry(emotion, topic):
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a human journaling assistant."},
            {"role": "user", "content": GEN_PROMPT.format(emotion=emotion, topic=topic)}
        ],
        "temperature": TEMPERATURE,
        "stream": False
    }
    try:
        res = requests.post(OLLAMA_URL, json=payload)
        raw = res.json()["message"]["content"].strip()
        return re.sub(r"<think>.*?</think>", "", raw, flags=re.DOTALL).strip()
    except Exception as e:
        print(f"❌ Error for {emotion}/{topic}: {e}")
        return None

def analyze_entry(text):
    ANALYSIS_PROMPT = """
You are a compassionate AI assistant trained to analyze journal entries.
Return ONLY a valid raw JSON object with the following keys:
  - summary: a one-sentence paraphrase of the overall entry
  - emotions: list of 1–3 dominant emotional states
  - patterns: list of up to 3 cognitive distortions (e.g., catastrophizing)
  - themes: list of up to 3 core life topics the entry relates to

Do NOT include explanations or markdown.
"""
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": ANALYSIS_PROMPT},
            {"role": "user", "content": text}
        ],
        "stream": False
    }
    try:
        res = requests.post(OLLAMA_URL, json=payload)
        content = res.json()["message"]["content"].strip()
        match = re.search(r"{.*}", content, re.DOTALL)
        return json.loads(match.group(0)) if match else None
    except Exception as e:
        print(f"❌ Analysis error: {e}")
        return None

def random_timestamp():
    start = datetime(2024, 1, 1)
    end = datetime(2024, 12, 31)
    delta = end - start
    offset = timedelta(days=random.randint(0, delta.days), hours=random.randint(18, 22), minutes=random.randint(0, 59))
    return (start + offset).isoformat()

def emotion_for_month(month):
    bias_map = {
        1: ["anxiety", "burnout"], 2: ["anxiety"], 3: ["confusion"], 4: ["hope"], 5: ["burnout"],
        6: ["joy", "contentment"], 7: ["joy"], 8: ["nostalgia"], 9: ["confusion", "anxiety"],
        10: ["nostalgia", "loneliness"], 11: ["loneliness"], 12: ["loneliness", "contentment"]
    }
    biased = bias_map.get(month, [])
    return random.choice(biased) if biased and random.random() < 0.6 else random.choice(emotions)

# Load existing
existing = []
if Path(OUTPUT_PATH).exists():
    with open(OUTPUT_PATH) as f:
        existing = [json.loads(line) for line in f if line.strip()]
existing_ts = set(e["timestamp"] for e in existing)

# Generate
new_entries = []
for _ in range(TOTAL_ENTRIES):
    ts = random_timestamp()
    if ts in existing_ts:
        continue
    emo = emotion_for_month(datetime.fromisoformat(ts).month)
    top = random.choice(topics)
    text = generate_entry(emo, top)
    if not text:
        continue
    analysis = analyze_entry(text)
    if not analysis:
        continue
    new_entries.append({
        "timestamp": ts,
        "text": text,
        "analysis": analysis
    })
    print(f"✅ {ts} — {emo} | {top}")

# Save all
with open(OUTPUT_PATH, "w") as f:
    for e in existing + new_entries:
        f.write(json.dumps(e) + "\n")

print(f"\n📄 Saved {len(new_entries)} new entries to: {OUTPUT_PATH}")


In [5]:
import os
import json

DATA_DIR = "/Users/Additional Storage/ML Projects/Journal app/data"

for filename in os.listdir(DATA_DIR):
    if filename.endswith(".jsonl") and not filename.startswith("rag_"):
        input_path = os.path.join(DATA_DIR, filename)
        output_path = os.path.join(DATA_DIR, f"rag_{filename}")

        rag_entries = []
        with open(input_path, "r") as f:
            for line in f:
                try:
                    entry = json.loads(line.strip())
                    rag_doc = {
                        "content": entry.get("text", ""),
                        "metadata": {
                            "timestamp": entry.get("timestamp"),
                            "themes": entry.get("analysis", {}).get("themes", []),
                            "emotions": entry.get("analysis", {}).get("emotions", [])
                        }
                    }
                    rag_entries.append(rag_doc)
                except json.JSONDecodeError:
                    print(f"❌ Skipped malformed line in {filename}")

        with open(output_path, "w") as f:
            for doc in rag_entries:
                f.write(json.dumps(doc) + "\n")

        print(f"✅ Created {output_path} with {len(rag_entries)} entries.")

✅ Created /Users/Additional Storage/ML Projects/Journal app/data/rag_journal_entries.jsonl with 1 entries.
✅ Created /Users/Additional Storage/ML Projects/Journal app/data/rag_test_generated_entries.jsonl with 119 entries.


In [7]:
import os

data_dir = "/Users/Additional Storage/ML Projects/Journal app/data"
jsonl_files = [f for f in os.listdir(data_dir) if f.endswith(".jsonl") and not f.startswith("rag_")]
print(jsonl_files)

['journal_entries.jsonl', 'test_generated_entries.jsonl']


In [9]:
import json

for file in jsonl_files:
    print(f"\n📂 Checking: {file}")
    path = os.path.join(data_dir, file)
    with open(path) as f:
        for i, line in enumerate(f):
            try:
                obj = json.loads(line.strip())
                if "timestamp" not in obj:
                    print(f"❌ Missing timestamp at line {i+1}: {obj}")
            except json.JSONDecodeError as e:
                print(f"⚠️ Malformed JSON at line {i+1}: {e}")


📂 Checking: journal_entries.jsonl

📂 Checking: test_generated_entries.jsonl


In [11]:
clean_entries = []
bad_entries = []

with open(os.path.join(data_dir, "journal_entries.jsonl")) as f:
    for i, line in enumerate(f):
        try:
            obj = json.loads(line.strip())
            if "timestamp" in obj:
                clean_entries.append(obj)
            else:
                bad_entries.append((i + 1, obj))
        except json.JSONDecodeError as e:
            bad_entries.append((i + 1, f"Malformed JSON: {e}"))

print(f"✅ {len(clean_entries)} clean entries")
print(f"❌ {len(bad_entries)} bad entries")

✅ 1 clean entries
❌ 0 bad entries


In [13]:
for file in jsonl_files:
    print(f"\n📁 Checking: {file}")
    path = os.path.join(data_dir, file)
    with open(path) as f:
        for i, line in enumerate(f):
            try:
                obj = json.loads(line.strip())
                if "timestamp" not in obj:
                    print(f"❌ {file} line {i+1} is missing timestamp:\n{obj}")
            except Exception as e:
                print(f"⚠️ Error in {file} line {i+1}: {e}")


📁 Checking: journal_entries.jsonl

📁 Checking: test_generated_entries.jsonl


In [15]:
entries = [json.loads(line.strip()) for line in open(path)]
entries.sort(key=lambda x: x["timestamp"], reverse=True)

In [17]:
import json
import os

# Step 1 – Point to the file that is causing the issue
path = "/Users/Additional Storage/ML Projects/Journal app/data/test_generated_entries.jsonl"  # or whichever is selected in the app

# Step 2 – Print each line and highlight whether 'timestamp' exists
bad_entries = []

print(f"🔍 Checking: {os.path.basename(path)}")
with open(path, "r") as f:
    for i, line in enumerate(f, 1):
        try:
            entry = json.loads(line.strip())
            has_timestamp = "timestamp" in entry
            print(f"Line {i:02d} → {'✅' if has_timestamp else '❌'} timestamp", end="")
            if not has_timestamp:
                print(f" | keys: {list(entry.keys())}")
                bad_entries.append((i, entry))
            else:
                print()
        except json.JSONDecodeError as e:
            print(f"❌ Line {i:02d} → JSON error: {e}")
            bad_entries.append((i, None))

# Step 3 – Summary
print(f"\nSummary: {len(bad_entries)} bad entries")

🔍 Checking: test_generated_entries.jsonl
Line 01 → ✅ timestamp
Line 02 → ✅ timestamp
Line 03 → ✅ timestamp
Line 04 → ✅ timestamp
Line 05 → ✅ timestamp
Line 06 → ✅ timestamp
Line 07 → ✅ timestamp
Line 08 → ✅ timestamp
Line 09 → ✅ timestamp
Line 10 → ✅ timestamp
Line 11 → ✅ timestamp
Line 12 → ✅ timestamp
Line 13 → ✅ timestamp
Line 14 → ✅ timestamp
Line 15 → ✅ timestamp
Line 16 → ✅ timestamp
Line 17 → ✅ timestamp
Line 18 → ✅ timestamp
Line 19 → ✅ timestamp
Line 20 → ✅ timestamp
Line 21 → ✅ timestamp
Line 22 → ✅ timestamp
Line 23 → ✅ timestamp
Line 24 → ✅ timestamp
Line 25 → ✅ timestamp
Line 26 → ✅ timestamp
Line 27 → ✅ timestamp
Line 28 → ✅ timestamp
Line 29 → ✅ timestamp
Line 30 → ✅ timestamp
Line 31 → ✅ timestamp
Line 32 → ✅ timestamp
Line 33 → ✅ timestamp
Line 34 → ✅ timestamp
Line 35 → ✅ timestamp
Line 36 → ✅ timestamp
Line 37 → ✅ timestamp
Line 38 → ✅ timestamp
Line 39 → ✅ timestamp
Line 40 → ✅ timestamp
Line 41 → ✅ timestamp
Line 42 → ✅ timestamp
Line 43 → ✅ timestamp
Line 44 → ✅ t

In [40]:
import json

rag_path = '/Users/Additional Storage/ML Projects/Journal app/data/rag/rag_journal_entries.jsonl'

with open(rag_path, "r") as f:
    lines = [json.loads(line) for line in f]

print(f"Total entries: {len(lines)}")
print("Last entry preview:", lines)

Total entries: 2
Last entry preview: [{'content': "I am checking if the new RAG works, I hope it does and le'ts see - I am nervous about it 123456789 practicing counting numbers", 'metadata': {'timestamp': '2025-06-04T22:36:40.885776', 'themes': ['anxiety about technology', 'practicing a routine task', 'new tool anxiety'], 'emotions': ['anxiety', 'nervousness']}}, {'content': "I am checking if the new RAG works, I hope it does and le'ts see - I am nervous about it 123456789 practicing counting numbers let's try something new may be that will workkk umm lets see RAG\nNLP", 'metadata': {'timestamp': '2025-06-04T22:38:19.492991', 'themes': ['testing RAG model', 'trying something new', 'experimentation with AI'], 'emotions': ['nervousness', 'uncertainty', 'excitement']}}]


In [66]:
# query_rag_journal.py

import os
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

# Path to vector store folder
RAG_FOLDER = "/Users/Additional Storage/ML Projects/Journal app/data/rag/rag_journal_entries"

# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Load FAISS index
db = FAISS.load_local(
    folder_path=RAG_FOLDER,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

def query_journal(question: str, k: int = 5):
    """
    Retrieve the most relevant journal entries for a given semantic query.
    Returns a list of (text, metadata) tuples.
    """
    results = db.similarity_search(question, k=k)
    return [(doc.page_content, doc.metadata) for doc in results]

# Example usage
q = "What are my recurring themes about self-worth?"
matches = query_journal(q)
for i, (text, meta) in enumerate(matches, 1):
    print(f"\nMatch {i}:")
    print("Text:", text)
    print("Metadata:", meta)


Match 1:
Text: I am checking if the new RAG works, I hope it does and le'ts see - I am nervous about it 123456789 practicing counting numbers let's try something new may be that will workkk umm lets see RAG
NLP
Metadata: {'timestamp': '2025-06-04T22:38:19.492991', 'themes': ['testing RAG model', 'trying something new', 'experimentation with AI'], 'emotions': ['nervousness', 'uncertainty', 'excitement']}

Match 2:
Text: I am checking if the new RAG works, I hope it does and le'ts see - I am nervous about it 123456789 practicing counting numbers
Metadata: {'timestamp': '2025-06-04T22:36:40.885776', 'themes': ['anxiety about technology', 'practicing a routine task', 'new tool anxiety'], 'emotions': ['anxiety', 'nervousness']}
